# A motivation for using Lie theory

This script shows four ways of simple interpolating.
The last is the valid lie theory interpolation as shown in the book found at Tussendrotten [Vslam](https://github.com/tussedrotten/vslam-handbook), p.36.


In [ ]:
import manifpy as manif
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import transforms3d as tf
%matplotlib widget

In [ ]:
def visualize_rotation(R, i):
    ax.quiver(0, 0, 0, R[0, 0], R[1, 0], R[2, 0], color='red', arrow_length_ratio=0.1, alpha=i)
    ax.quiver(0, 0, 0, R[0, 1], R[1, 1], R[2, 1], color='green', arrow_length_ratio=0.1, alpha=i)
    ax.quiver(0, 0, 0, R[0, 2], R[1, 2], R[2, 2], color='blue', arrow_length_ratio=0.1, alpha=i)
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

A simple example showing how one can't interpolate between two matrix representations. 

$$
\mathbb{R}_i = \mathbb{R}_s + i \cdot (\mathbb{R}_f - \mathbb{R}_s)
$$
Where $i \in [0, 1]$

This is of course not valid, since $\mathbb{R}_j \not \in SO3 \; \text{for some} \; j \in [0, 1] $ 

In [ ]:
Matrix_start = tf.euler.euler2mat(0,0,0)
Matrix_end = tf.euler.euler2mat(np.pi/3, -np.pi/4, np.pi/2)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in np.arange(0, 1, 0.1):
    intermediate = Matrix_start + (Matrix_end - Matrix_start) * i
    visualize_rotation(intermediate, i)
ax.set_title('Using Rotation matrix interpolation')
plt.show()


Can also interpolate between two euler angle representations. This is better 

$$
\boldsymbol{\theta}_i = \boldsymbol{\theta}_s + i \cdot (\boldsymbol{\theta}_f - \boldsymbol{\theta}_s)
$$
Where $i \in [0, 1]$

In [ ]:
Euler_start = np.array([0,0,0])
Euler_end = np.array([np.pi/3, -np.pi/4, np.pi/2])

interpol_eul = []

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in np.arange(0, 1.1, 0.1):
    intermediate = Euler_start + (Euler_end - Euler_start) * i
    rotmat = tf.euler.euler2mat(intermediate[0], intermediate[1], intermediate[2])
    visualize_rotation(rotmat, i)
    interpol_eul.append(rotmat)
ax.set_title('Using Euler angle interpolation')
plt.show()

Can also interpolate between two quaternion representations. This is even better 

$$
\boldsymbol{q}_i = \boldsymbol{q}_s + i \cdot (\boldsymbol{q}_f - \boldsymbol{q}_s)
$$
Where $i \in [0, 1]$

In [ ]:
quat_start = tf.euler.euler2quat(0,0,0)
quat_end = tf.euler.euler2quat(np.pi/3, -np.pi/4, np.pi/2)

interpol_quat = []

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in np.arange(0, 1.1, 0.1):
    intermediate = quat_start + (quat_end - quat_start) * i
    rotmat = tf.euler.quat2mat(intermediate)
    visualize_rotation(rotmat, i)
    interpol_quat.append(rotmat)
ax.set_title('Using Quaternion interpolation')
plt.show()

The correct way would be to use the special operators from lie theory

$$
\boldsymbol{\theta}_i = \boldsymbol{\theta}_s \oplus i \cdot (\boldsymbol{\theta}_f \ominus \boldsymbol{\theta}_s)
$$
Where $i \in [0, 1]$

In [ ]:
Matrix_start = manif.SO3.Identity()
Matrix_end = manif.SO3(np.pi/3, -np.pi/4, np.pi/2)
interpol_lie = []
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in np.arange(0, 1.1, 0.1):
    intermediate = Matrix_start.plus(i * Matrix_end.minus(Matrix_start))
    visualize_rotation(intermediate.rotation(), i)
    interpol_lie.append(intermediate.rotation())
ax.set_title('Using interpolation based on lie theory')
plt.show()

We can now show how wrong our interpolation is by comparing the interpolated matrix with the correct lie theory interpolation.
For metric we have used the L2 norm of the residual between the Rotation matrices

In [ ]:
plt.figure()
residual_euler = np.array(interpol_lie) - np.array(interpol_eul)
residual_quat = np.array(interpol_lie) - np.array(interpol_quat)

e_eul = [np.linalg.norm(res) for res in residual_euler]
e_quat = [np.linalg.norm(res) for res in residual_quat]

plt.plot(e_eul, label="euler")
plt.plot(e_quat, label="quaternion")
plt.legend()
plt.title("Pose error")
plt.xlabel("iteration")
plt.ylabel("residual error")

plt.show()

This shows that the quaternion is closer to the correct value. Also we can see that the error is largest for the euler angle representation in the middle. This makes sense since the euler angle representation is not a good representation for interpolation. We can also plot the two matrices for seeing the difference

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_title("biggest difference between euler- and lie group interpolation")
R = interpol_eul[5]
ax.quiver(0, 0, 0, R[0, 0], R[1, 0], R[2, 0], color='red', arrow_length_ratio=0.1, alpha=1)
ax.quiver(0, 0, 0, R[0, 1], R[1, 1], R[2, 1], color='red', arrow_length_ratio=0.1, alpha=1)
ax.quiver(0, 0, 0, R[0, 2], R[1, 2], R[2, 2], color='red', arrow_length_ratio=0.1, alpha=1, label="euler")
R = interpol_lie[5]
ax.quiver(0, 0, 0, R[0, 0], R[1, 0], R[2, 0], color='blue', arrow_length_ratio=0.1, alpha=1)
ax.quiver(0, 0, 0, R[0, 1], R[1, 1], R[2, 1], color='blue', arrow_length_ratio=0.1, alpha=1)
ax.quiver(0, 0, 0, R[0, 2], R[1, 2], R[2, 2], color='blue', arrow_length_ratio=0.1, alpha=1, label="lie")
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
plt.legend()
plt.show()

This shows how important it is to use proper rigorous math when interpolating between two rotations.